In [1]:
import emcee
import json
import pandas as pd
import exohammer as exo

# Load measurement data
rv_df = pd.read_csv('./example_data/RV_kepler-36.csv')
k36b_ttv_df = pd.read_csv('./example_data/TTV_kepler-36b.csv')
k36c_ttv_df = pd.read_csv('./example_data/TTV_kepler-36c.csv')

# Load priors for orbital elements and stellar mass
with open('./example_data/orbital_elements.json', 'r') as f:
    orbital_elements = json.load(f)
    m_star = orbital_elements.pop('m_star')

orbital_elements

{'mass_b': [3.0034893488507934e-06, 3.0034893488507933e-05],
 'period_b': [13.799, 13.899000000000001],
 'eccentricity_b': [0.0, 0.04],
 'inclination_b': [90],
 'longnode_b': [0.0],
 'argument_b': [0.0, 360.0],
 'mean_anomaly_b': [0.0, 360.0],
 'mass_c': [3.0034893488507934e-06, 3.0034893488507933e-05],
 'period_c': [16.186899999999998, 16.2869],
 'eccentricity_c': [0.0, 0.04],
 'inclination_c': [90],
 'longnode_c': [0.0],
 'argument_c': [0.0, 360.0],
 'mean_anomaly_c': [0.0, 360.0],
 'mass_d': [3.0034893488507934e-06, 0.0012013957395403173],
 'period_d': [40, 600],
 'eccentricity_d': [0.0],
 'inclination_d': [90.0],
 'longnode_d': [0.0],
 'argument_d': [0.0],
 'mean_anomaly_d': [0.0, 360.0]}

In [2]:
kepler_36 = exo.planetary_system.PlanetarySystem(nplanets_ttvs=2, 
                                                 nplanets_rvs=3, 
                                                 orbital_elements=orbital_elements)

data = exo.data.Data(m_star)

data.add_ttv(planet_label = 'b',
             epoch = k36b_ttv_df['epoch'].astype(int).to_list(),
             measured = k36b_ttv_df['Transit Time'].to_list(),
             error = k36b_ttv_df['error'].to_list())

data.add_ttv(planet_label = 'c',
             epoch = k36c_ttv_df['epoch'].astype(int).to_list(),
             measured = k36c_ttv_df['Transit Time'].to_list(),
             error = k36c_ttv_df['error'].to_list())

data.add_rvs(bjds = rv_df['BJD'].to_list(),
             velocities= rv_df['mean_velocity'].to_list(),
             error = rv_df['velocity_error'].to_list())
run = exo.mcmc_run.MCMCRun(kepler_36, data)


In [3]:
run.explore_iteratively(total_iterations=10000000, 
                        checkpoints=1000, 
                        burnin_factor=.2, 
                        thinning_factor=.001,
	                    moves=[(emcee.moves.DEMove(live_dangerously=True), 0.9), 
                               (emcee.moves.DESnookerMove(live_dangerously=True), 0.1)],
	                    verbose=True, 
                        tune=True, 
                        silent=True)

Steps completed: 0
Run 0 of 10000, 1000 steps


  0%|          | 0/1000 [00:00<?, ?it/s]/opt/anaconda3/envs/exo_env/lib/python3.8/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]
100%|██████████| 1000/1000 [00:29<00:00, 34.17it/s]

[29.667503250000003]


NameError: name 'np' is not defined

In [ ]:
run.autocorr()

/opt/anaconda3/envs/exo_env/lib/python3.8/site-packages/emcee/autocorr.py:36: RuntimeWarning: invalid value encountered in true_divide
  acf /= acf[0]


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])